In [1]:
import os
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import torch
from typing import Dict,List,Tuple

In [19]:
## Simulate the args like in the `main_*.py` files
class ARGS:
    # federated arguments
    # epochs:int = 1000         # rounds of training
    epochs: int = 10  # rounds of training
    train_test_same: int = 0  # use same testing for
    num_users: int = 100  # number of users: K
    shard_per_user: int = 2  # classes per user
    frac: float = 0.1  # the fraction of clients: C
    local_ep: int = 1  # the number of local epochs: E
    local_bs: int = 10  # local batch size: B
    bs: int = 128  # test batch size
    lr: float = 0.01  # learning rate
    # results_save:str = "run1"
    momentum: float = 0.5  # SGD momentum (default: 0.5)
    # gpu:int = 0
    split: str = "user"  # train-test split type, user or sample
    # grad_norm:str           # use_gradnorm_avging
    local_ep_pretrain: int = 0  # the number of pretrain local ep
    lr_decay: float = 1.0  # learning rate decay per round

    # model arguments
    model: str = "cnn"  # model name
    kernel_num: int = 9  # number of each kind of kernel
    kernel_sizes: str = "3,4,5"  # comma-separated kernel size to use for convolution
    norm: str = "batch_norm"  # batch_norm, layer_norm, or None
    num_filters: int = 32  # number of filters for conv nets
    max_pool: str = True  # whether use max pooling rather than strided convolutions
    num_layers_keep: int = 1  # number layers to keep

    # other arguments
    dataset: str = "coba"  # name of dataset
    log_level: str = "warning"  # level of logger
    iid: bool = True  # "store_true" #whether iid or not
    num_classes: int = 14  # number of classes
    num_channels: int = 3  # number of channels of images RGB
    gpu: int = 0  # GPU ID, -1 for CPU
    stopping_rounds: int = 10  # rounds of early stopping
    verbose: bool = True  # "store_true"
    print_freq: int = 100  # print loss frequency during training
    seed: int = 1  # random seed (default:1)
    test_freq: int = 1  # how often to test on val set
    load_fed: str = ""  # define pretrained federated model path
    results_save: str = "run1"  # define fed results save folder
    start_saving: int = 0  # when to start saving models


args = ARGS()

args.device = torch.device(
    "cuda:{}".format(args.gpu)
    if torch.cuda.is_available() and args.gpu != -1
    else "cpu"
)

args.num_users, args.device

(100, device(type='cuda', index=0))

## MNIST

In [20]:
# args.num_users = 98
# args.frac = "0.3"
# args.results_save = "coba_fedavg_bestcase_run2"
# args.frac = "0.5";args.results_save = "coba_fedavg_averagecase_run2"
# args.frac = "1.0";args.results_save = "coba_fedavg_worstcase_run2"
args.iid = False
args.log_level = "info"  # "debug" might crash the notebook
args.dataset = "mnist"

In [21]:
# experiment_run_dir: Path = Path(Path.cwd().parent, "save", "coba_legacy")
AS_PDF:bool = True
experiment_run_dir: Path = Path(Path.cwd().parent, "save", args.dataset)
async_fl_scenarios: Dict[str, str] = {"0.3": "best", "0.5": "average", "1.0": "worst"}
experiment_run_dir

PosixPath('/home/tank/Coding-Practice/objective3-fl-experiment/dynamic-model-selection-for-async-fl/save/mnist')

In [22]:
# FOR MNIST
# experiment_seed in (0,1,2,3,4,5,6,7,8,9,10,13)
# scenario in ("best","average","worst"):

# experiment_seed = 0
# scenario = "best"
seeds: Tuple[int] = (0,) # if not LEGACY else (0,1,2,3,4,5,6,7,8,9,10,13)
# for experiment_seed in (0,1,2,3,4,5,6,7,8,9,10,13):
for experiment_seed in seeds:
    scenarios_experiment_csv_files:List[Tuple[str,Path]] = []
    
    for scenario in ("best","average","worst"):
        base_dir = None
        for dir in experiment_run_dir.glob("*"):
            scenario_percent: str = (
                dir.as_posix().split(os.sep)[-1].split("_")[-2].replace("C", "")
            )
            # print(f"\t{scenario_percent} -> {async_fl_scenarios[scenario_percent]}")
            
            if async_fl_scenarios[scenario_percent] == scenario:
                chosen_scenario_dir = dir
            
        all_chosen_scenario_runs_dir: Path = Path(chosen_scenario_dir, "shard2")
        for dir in all_chosen_scenario_runs_dir.glob("*"):
            seed: int = int(dir.as_posix().split(os.sep)[-1].split("_")[0].split("d")[-1])
        
            if experiment_seed == seed:
                base_dir = Path(dir)
        
        # Raise error if no directory is found with the given SEED
        if base_dir is None:
            raise FileNotFoundError(f"Directory with the provided seed '{experiment_seed}' does not exist. Please choose a different one and try again.")
        scenario_results_csv:Path = Path(base_dir, "fed","results.csv")
        # print(f"{scenario_results_csv.as_posix()} exists: {scenario_results_csv.exists()}")
        scenarios_experiment_csv_files.append((scenario.title(), scenario_results_csv))
    
    dfs:List[pd.DataFrame] = []
    
    for scenario, filepath in scenarios_experiment_csv_files:
        df = pd.read_csv(filepath, delimiter=",")
        df = df.assign(scenario=[scenario]*len(df))
        dfs.append(df)
    
    scenario_graphs_dir: Path = Path(Path.cwd(), args.dataset, f"seed{experiment_seed}_scenario_metrics")
    scenario_graphs_dir.mkdir(exist_ok=True, parents=True)
    
    col_names:List[str] = [col_name for col_name in dfs[0].columns.values if col_name not in ("epoch","scenario")]
    plot_titles:Dict[str,str] = {
        "acc":"Accuracy",
        "f1":"F1-Score",
        "loss_avg":"Average Loss",
    }
    
    for col_name in col_names:
        # graph_save_file: Path = Path(scenario_graphs_dir, f"{col_name.title()}.png")
        graph_save_file: Path = Path(scenario_graphs_dir, f"{col_name.title()}.{'pdf' if AS_PDF else 'png'}")
        
        # fig, ax = plt.subplots(figsize=(10, 10))
        fig, ax = plt.subplots(figsize=(5, 5))
        palettes = [sns.color_palette("bright",n_colors=1),sns.color_palette("Dark2",n_colors=1),sns.color_palette("magma",n_colors=1)]
        markers = ["v","P","d"]
        
        for palette, marker, df in zip(palettes,markers,dfs):
            epochs = np.arange(len(df) // 10) * 10
            adjusted_col = df.groupby(np.arange(len(df)) // 10)[col_name].mean().values  # averaged every 10 epochs
            hue = df["scenario"].values[:100]

            plot_title:str = plot_titles.get(col_name.replace("_test",""), col_name.replace("_test","")).title()

            sns.set_style("ticks", {"grid.linestyle":"-."})
            # sns.set_style("dark", {"grid.color":"0.6","grid.linestyle":"-."})


            # graph line (0.7 opacity)
            sns.lineplot(x=epochs,
                         y=adjusted_col,
                         hue=hue, 
                         palette=palette, 
                         legend=True,
                         alpha=0.7,
                         ax=ax,
            ).set(
                            title=plot_title,
                            xlabel="Epochs",
                            ylabel="Value"
            )
            
            # graph markers (no opacity)
            lineplot_markers = sns.lineplot(x=epochs,
                         y=adjusted_col,
                         linestyle="",
                         marker=marker,
                         markerfacecolor=palette[0],
                         # hue=hue,
                         # palette=palette,
                         markersize=4,
                         legend=True,
                         alpha=1,
                         ax=ax
            )

            plt.legend(frameon=True, framealpha=1).get_frame().set_facecolor("white")

            for handler, legend_marker in zip(lineplot_markers.legend_.legend_handles, markers):
                handler.set_marker(legend_marker)

            
            plt.grid()
            
            plt.savefig(graph_save_file)

        plt.close()
            # plt.clf()

## CIFAR10

In [3]:
# args.num_users = 98
# args.frac = "0.3"
# args.results_save = "coba_fedavg_bestcase_run2"
# args.frac = "0.5";args.results_save = "coba_fedavg_averagecase_run2"
# args.frac = "1.0";args.results_save = "coba_fedavg_worstcase_run2"
args.iid = False
args.log_level = "info"  # "debug" might crash the notebook
args.dataset = "cifar10"

In [4]:
# experiment_run_dir: Path = Path(Path.cwd().parent, "save", "coba_legacy")
AS_PDF:bool = True
experiment_run_dir: Path = Path(Path.cwd().parent, "save", args.dataset)
async_fl_scenarios: Dict[str, str] = {"0.3": "best", "0.5": "average", "1.0": "worst"}
experiment_run_dir

PosixPath('/home/tank/Coding-Practice/objective3-fl-experiment/dynamic-model-selection-for-async-fl/save/cifar10')

In [8]:
# FOR CIFAR10
# experiment_seed in (0,1,2,3,4,5,6,7,8,9,10,13)
# scenario in ("best","average","worst"):

# experiment_seed = 0
# scenario = "best"
seeds: Tuple[int] = (10,) # if not LEGACY else (0,1,2,3,4,5,6,7,8,9,10,13)
# for experiment_seed in (0,1,2,3,4,5,6,7,8,9,10,13):
for experiment_seed in seeds:
    scenarios_experiment_csv_files:List[Tuple[str,Path]] = []
    
    for scenario in ("best","average","worst"):
        base_dir = None
        for dir in experiment_run_dir.glob("*"):
            scenario_percent: str = (
                dir.as_posix().split(os.sep)[-1].split("_")[-2].replace("C", "")
            )
            # print(f"\t{scenario_percent} -> {async_fl_scenarios[scenario_percent]}")
            
            if async_fl_scenarios[scenario_percent] == scenario:
                chosen_scenario_dir = dir
            
        all_chosen_scenario_runs_dir: Path = Path(chosen_scenario_dir, "shard2")
        for dir in all_chosen_scenario_runs_dir.glob("*"):
            seed: int = int(dir.as_posix().split(os.sep)[-1].split("_")[0].split("d")[-1])
        
            if experiment_seed == seed:
                base_dir = Path(dir)
        
        # Raise error if no directory is found with the given SEED
        if base_dir is None:
            raise FileNotFoundError(f"Directory with the provided seed '{experiment_seed}' does not exist. Please choose a different one and try again.")
        scenario_results_csv:Path = Path(base_dir, "fed","results.csv")
        # print(f"{scenario_results_csv.as_posix()} exists: {scenario_results_csv.exists()}")
        scenarios_experiment_csv_files.append((scenario.title(), scenario_results_csv))
    
    dfs:List[pd.DataFrame] = []
    
    for scenario, filepath in scenarios_experiment_csv_files:
        df = pd.read_csv(filepath, delimiter=",")
        df = df.assign(scenario=[scenario]*len(df))
        dfs.append(df)
    
    scenario_graphs_dir: Path = Path(Path.cwd(), args.dataset, f"seed{experiment_seed}_scenario_metrics")
    scenario_graphs_dir.mkdir(exist_ok=True, parents=True)
    
    col_names:List[str] = [col_name for col_name in dfs[0].columns.values if col_name not in ("epoch","scenario")]
    plot_titles:Dict[str,str] = {
        "acc":"Accuracy",
        "f1":"F1-Score",
        "loss_avg":"Average Loss",
    }
    
    for col_name in col_names:
        # graph_save_file: Path = Path(scenario_graphs_dir, f"{col_name.title()}.png")
        graph_save_file: Path = Path(scenario_graphs_dir, f"{col_name.title()}.{'pdf' if AS_PDF else 'png'}")
        
        # fig, ax = plt.subplots(figsize=(10, 10))
        fig, ax = plt.subplots(figsize=(5, 5))
        palettes = [sns.color_palette("bright",n_colors=1),sns.color_palette("Dark2",n_colors=1),sns.color_palette("magma",n_colors=1)]
        markers = ["v","P","d"]
        
        for palette, marker, df in zip(palettes,markers,dfs):
            epochs = np.arange(len(df) // 10) * 10
            adjusted_col = df.groupby(np.arange(len(df)) // 10)[col_name].mean().values  # averaged every 10 epochs
            hue = df["scenario"].values[:100]

            plot_title:str = plot_titles.get(col_name.replace("_test",""), col_name.replace("_test","")).title()

            sns.set_style("ticks", {"grid.linestyle":"-."})
            # sns.set_style("dark", {"grid.color":"0.6","grid.linestyle":"-."})


            # graph line (0.7 opacity)
            sns.lineplot(x=epochs,
                         y=adjusted_col,
                         hue=hue, 
                         palette=palette, 
                         legend=True,
                         alpha=0.7,
                         ax=ax,
            ).set(
                            title=plot_title,
                            xlabel="Epochs",
                            ylabel="Value"
            )
            
            # graph markers (no opacity)
            lineplot_markers = sns.lineplot(x=epochs,
                         y=adjusted_col,
                         linestyle="",
                         marker=marker,
                         markerfacecolor=palette[0],
                         # hue=hue,
                         # palette=palette,
                         markersize=4,
                         legend=True,
                         alpha=1,
                         ax=ax
            )

            plt.legend(frameon=True, framealpha=1).get_frame().set_facecolor("white")

            for handler, legend_marker in zip(lineplot_markers.legend_.legend_handles, markers):
                handler.set_marker(legend_marker)

            
            plt.grid()
            
            plt.savefig(graph_save_file)

        plt.close()
            # plt.clf()

## COBA

In [13]:
args.num_users = 98
# args.frac = "0.3"
# args.results_save = "coba_fedavg_bestcase_run2"
# args.frac = "0.5";args.results_save = "coba_fedavg_averagecase_run2"
# args.frac = "1.0";args.results_save = "coba_fedavg_worstcase_run2"
args.iid = False
args.log_level = "info"  # "debug" might crash the notebook
args.seed = 0
args.dataset = "coba"

# base_dir: Path = Path(
#     Path.cwd().parent,
#     "save",
#     "coba_legacy",
#     f"{args.model}_iid{args.iid}_num{args.num_users}_C{args.frac}_le{args.local_ep}",
#     f"shard{args.shard_per_user}",
# )

# base_dir = Path(base_dir, f"seed{args.seed}_{args.results_save}")
# # coba_results_df: pd.DataFrame = pd.read_csv(coba_results_path, delimiter=",")
# # coba_results_df
# base_dir

In [17]:
# experiment_run_dir: Path = Path(Path.cwd().parent, "save", "coba_legacy")
LEGACY:bool = True
AS_PDF:bool = True
experiment_run_dir: Path = Path(Path.cwd().parent, "save", args.dataset if not LEGACY else "coba_legacy")
async_fl_scenarios: Dict[str, str] = {"0.3": "best", "0.5": "average", "1.0": "worst"}
experiment_run_dir

PosixPath('/home/tank/Coding-Practice/objective3-fl-experiment/dynamic-model-selection-for-async-fl/save/coba')

In [18]:
# FOR COBA
# experiment_seed in (0,1,2,3,4,5,6,7,8,9,10,13)
# scenario in ("best","average","worst"):

# experiment_seed = 0
# scenario = "best"
seeds: Tuple[int] = (250,750) if not LEGACY else (0,1,2,3,4,5,6,7,8,9,10,13)
# for experiment_seed in (0,1,2,3,4,5,6,7,8,9,10,13):
for experiment_seed in seeds:
    scenarios_experiment_csv_files:List[Tuple[str,Path]] = []
    
    for scenario in ("best","average","worst"):
        base_dir = None
        for dir in experiment_run_dir.glob("*"):
            scenario_percent: str = (
                dir.as_posix().split(os.sep)[-1].split("_")[-2].replace("C", "")
            )
            # print(f"\t{scenario_percent} -> {async_fl_scenarios[scenario_percent]}")
            
            if async_fl_scenarios[scenario_percent] == scenario:
                chosen_scenario_dir = dir
            
        all_chosen_scenario_runs_dir: Path = Path(chosen_scenario_dir, "shard2")
        for dir in all_chosen_scenario_runs_dir.glob("*"):
            seed: int = int(dir.as_posix().split(os.sep)[-1].split("_")[0].split("d")[-1])
        
            if experiment_seed == seed:
                base_dir = Path(dir)
        
        # Raise error if no directory is found with the given SEED
        if base_dir is None:
            raise FileNotFoundError(f"Directory with the provided seed '{experiment_seed}' does not exist. Please choose a different one and try again.")
        scenario_results_csv:Path = Path(base_dir, "fed","results.csv")
        # print(f"{scenario_results_csv.as_posix()} exists: {scenario_results_csv.exists()}")
        scenarios_experiment_csv_files.append((scenario.title(), scenario_results_csv))
    
    dfs:List[pd.DataFrame] = []
    
    for scenario, filepath in scenarios_experiment_csv_files:
        df = pd.read_csv(filepath, delimiter=",")
        df = df.assign(scenario=[scenario]*len(df))
        dfs.append(df)
    
    scenario_graphs_dir: Path = Path(Path.cwd(), args.dataset, f"seed{experiment_seed}_scenario_metrics")
    scenario_graphs_dir.mkdir(exist_ok=True, parents=True)
    
    col_names:List[str] = [col_name for col_name in dfs[0].columns.values if col_name not in ("epoch","scenario")]
    plot_titles:Dict[str,str] = {
        "acc":"Accuracy",
        "f1":"F1-Score",
        "loss_avg":"Average Loss",
    }
    
    for col_name in col_names:
        # graph_save_file: Path = Path(scenario_graphs_dir, f"{col_name.title()}.png")
        graph_save_file: Path = Path(scenario_graphs_dir, f"{col_name.title()}.{'pdf' if AS_PDF else 'png'}")
        
        # fig, ax = plt.subplots(figsize=(10, 10))
        fig, ax = plt.subplots(figsize=(5, 5))
        palettes = [sns.color_palette("bright",n_colors=1),sns.color_palette("Dark2",n_colors=1),sns.color_palette("magma",n_colors=1)]
        markers = ["v","P","d"]
        
        for palette, marker, df in zip(palettes,markers,dfs):
            epochs = np.arange(len(df) // 10) * 10
            adjusted_col = df.groupby(np.arange(len(df)) // 10)[col_name].mean().values  # averaged every 10 epochs
            hue = df["scenario"].values[:100]

            plot_title:str = plot_titles.get(col_name.replace("_test",""), col_name.replace("_test","")).title()

            sns.set_style("ticks", {"grid.linestyle":"-."})
            # sns.set_style("dark", {"grid.color":"0.6","grid.linestyle":"-."})


            # graph line (0.3 opacity)
            sns.lineplot(x=epochs,
                         y=adjusted_col,
                         hue=hue, 
                         palette=palette, 
                         legend=True,
                         alpha=0.7,
                         ax=ax,
            ).set(
                            title=plot_title,
                            xlabel="Epochs",
                            ylabel="Value"
            )
            
            # graph markers (no opacity)
            lineplot_markers = sns.lineplot(x=epochs,
                         y=adjusted_col,
                         linestyle="",
                         marker=marker,
                         markerfacecolor=palette[0],
                         # hue=hue,
                         # palette=palette,
                         markersize=4,
                         legend=True,
                         alpha=1,
                         ax=ax
            )

            plt.legend(frameon=True, framealpha=1).get_frame().set_facecolor("white")

            for handler, legend_marker in zip(lineplot_markers.legend_.legend_handles, markers):
                handler.set_marker(legend_marker)

            
            plt.grid()
            
            plt.savefig(graph_save_file)

        plt.close()
            # plt.clf()

## Misc

In [9]:
dfs:List[pd.DataFrame] = []

for scenario, filepath in scenarios_experiment_csv_files:
    df = pd.read_csv(filepath, delimiter=",")
    df = df.assign(scenario=[scenario]*len(df))
    dfs.append(df)

In [34]:
# from matplotlib import colormaps
# list(colormaps)

In [8]:
sns.color_palette("bright",n_colors=1)

[(0.00784313725490196, 0.24313725490196078, 1.0)]

In [14]:
# sns.color_palette("Accent",n_colors=1)
sns.color_palette(palette='Dark2',n_colors=1)

[(0.10588235294117647, 0.6196078431372549, 0.4666666666666667)]

In [13]:
sns.color_palette("magma",n_colors=1)

[(0.716387, 0.214982, 0.47529)]

In [25]:
for name in [col_name for col_name in dfs[0].columns.values if col_name not in ("epoch","scenario")]:
    print(name.replace("_test",""))
# [col_name for col_name in dfs[0].columns.values if col_name not in ("epoch","scenario")]

loss_avg
loss
acc
f1
precision
recall
best_acc


In [4]:
scenario_graphs_dir: Path = Path(Path.cwd(), f"seed{experiment_seed}_scenario_metrics")
scenario_graphs_dir.mkdir(exist_ok=True)

col_names:List[str] = [col_name for col_name in dfs[0].columns.values if col_name not in ("epoch","scenario")]
for col_name in col_names:
    graph_save_file: Path = Path(scenario_graphs_dir, f"{col_name.title()}.png")
    
    fig, ax = plt.subplots(figsize=(5, 5))
    palettes = [sns.color_palette("bright",n_colors=1),sns.color_palette("Accent",n_colors=1),sns.color_palette("magma",n_colors=1)]
    
    for palette, df in zip(palettes,dfs):
        epochs = np.arange(len(df) // 10) * 10
        adjusted_col = df.groupby(np.arange(len(df)) // 10)[col_name].mean().values  # averaged every 10 epochs
        hue = df["scenario"].values[:100]
    # avg_adjusted_col = (average_scenario_df.groupby(np.arange(len(average_scenario_df)) // 10).mean()[col_name].values)  # averaged every 10 epochs
    # worst_adjusted_col = (worst_scenario_df.groupby(np.arange(len(worst_scenario_df)) // 10).mean()[col_name].values)  # averaged every 10 epochs
        sns.set_style("ticks", {"grid.linestyle":"-."})

        sns.lineplot(x=epochs,y=adjusted_col, hue=hue, palette=palette, ax=ax).set(title=col_name.title(),xlabel="Epochs",ylabel="Value")
        plt.grid()
        # plt.savefig(graph_save_file)
        # plt.clf()

NameError: name 'experiment_seed' is not defined

In [1]:
# experiment_run_dir: Path = Path(Path.cwd().parent, "save", "coba_legacy")
# async_fl_scenarios: Dict[str, str] = {"0.3": "best", "0.5": "average", "1.0": "worst"}

# for experiment_seed in (0,1,2,3,4,5,6,7,8,9,10,13):
#     for scenario in ("best","average","worst"):
#         for dir in experiment_run_dir.glob("*"):
#             scenario_percent: str = (
#                 dir.as_posix().split(os.sep)[-1].split("_")[-2].replace("C", "")
#             )
#             # print(f"\t{scenario_percent} -> {async_fl_scenarios[scenario_percent]}")
        
#             if async_fl_scenarios[scenario_percent] == scenario:
#                 chosen_scenario_dir = dir

#         all_chosen_scenario_runs_dir: Path = Path(chosen_scenario_dir, "shard2")
#         for dir in all_chosen_scenario_runs_dir.glob("*"):
#             seed: int = int(dir.as_posix().split(os.sep)[-1].split("_")[0].split("d")[-1])
        
#             if experiment_seed == seed:
#                 base_dir = Path(dir)
        
#         # Raise error if no directory is found with the given SEED
#         if base_dir is None:
#             raise FileNotFoundError(f"Directory with the provided seed '{SEED}' does not exist. Please choose a different one and try again.")

# TODO: find way to graph results across each scenario (with the SAME seed) together

In [ ]:
def do_all_dynamic_model_selection(args):
    # experiment_run_dir: Path = Path(Path.cwd(), "save", "coba_legacy")
    experiment_run_dir: Path = Path(Path.cwd().parent, "save", "coba_legacy")
    async_fl_scenarios: Dict[str, str] = {"0.3": "best", "0.5": "average", "1.0": "worst"}

    for experiment_seed in (0,1,2,3,4,5,6,7,8,9,10,13):
        for scenario in ("best","average","worst"):
            for dir in experiment_run_dir.glob("*"):
                scenario_percent: str = (
                    dir.as_posix().split(os.sep)[-1].split("_")[-2].replace("C", "")
                )
                # print(f"\t{scenario_percent} -> {async_fl_scenarios[scenario_percent]}")
            
                if async_fl_scenarios[scenario_percent] == scenario:
                    chosen_scenario_dir = dir

            all_chosen_scenario_runs_dir: Path = Path(chosen_scenario_dir, "shard2")
            for dir in all_chosen_scenario_runs_dir.glob("*"):
                seed: int = int(dir.as_posix().split(os.sep)[-1].split("_")[0].split("d")[-1])
            
                if experiment_seed == seed:
                    base_dir = Path(dir)
            
            # Raise error if no directory is found with the given SEED
            if base_dir is None:
                raise FileNotFoundError(f"Directory with the provided seed '{SEED}' does not exist. Please choose a different one and try again.")
            
            print(f"Starting with {base_dir.as_posix()} (seed '{experiment_seed}')")
            # dynamic_model_selector_and_saver(args=args, base_dir=base_dir)
            print(f"Finished with {base_dir.as_posix()} (seed '{experiment_seed}')")